<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
path = "drive/My Drive/news/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall("train")
path = "drive/My Drive/news/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall("test")

In [ ]:
import os
import pandas as pd
import glob
def getdata(base):
    contents = []
    types = []
    for dir in glob.glob(os.path.join(base, "*")):
        t = os.path.split(dir)[-1]
        for fn in glob.glob(os.path.join(dir, "*.txt")) + glob.glob(os.path.join(dir, "*.TXT")):
            with open(fn) as f:
                contents.append(f.read())
                types.append(t)
    df = pd.DataFrame({
        "content":contents,
        "ans":types
    })
    return df
train_df = getdata("train/chinese_news_trans")
train_df
        

In [ ]:
test_df = getdata("test/chinese_news_test")
test_df

In [ ]:
# 答案: Series.replace({"李白":0, "杜甫":1})
# unique/value_counts
u = train_df["ans"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)
y_test

In [ ]:
# Series.apply(func)
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(poemcut)
x_test = test_df["content"].apply(poemcut)
x_train

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

In [ ]:
p = input("輸入一首新聞:")
count = vec.transform([poemcut(p)])
proba = clf.predict_proba(count)[0]
for name, prob in zip(u, proba):
    print(name, "的機率:", round(prob, 2))
# argmax: 找最大值的index
print("成分最高;", u[proba.argmax()])